In [1]:
%config InteractiveShell.ast_node_interactivity = 'all' # 'all', 'last', 'last_expr' or 'none', 'last_expr_or_assign'
# %config InlineBackend.figure_format = 'svg'


In [5]:
import numpy as np
import gym
from envs.cleanup import CleanupEnv
from envs.cleanup import CleanupAgent

# self, agent_id, start_pos, start_orientation, grid, row_size, col_size

env = 'cleanup-v0'
# try:
#     gym.register(env, entry_point=CleanupEnv, kwargs={'num_agents':10})
# except:
#     raise Exception('environment registered')
env = gym.make(env)
env.seed



<bound method Env.seed of <envs.cleanup.CleanupEnv object at 0x7f03490b70f0>>

In [2]:
env.num_agents

10

In [6]:
env.seed()

In [2]:
from envs.harvest import HarvestEnv
from envs.cleanup import CleanupEnv
from algo.ppo import PPO
from algo.model import Policy
from algo.storage import RolloutStorage
# from config import get_config
from envs.vec_env import SubprocVecEnv, DummyVecEnv
from algo.algo_utils import update_linear_schedule

In [3]:
def make_parallel_env(args):
    def get_env_fn(rank):
        def init_env():
            if args['env_name'] == "cleanup":
                env = CleanupEnv(num_agents=args['num_agents'])
            elif args['env_name'] == "harvest":
                env = HarvestEnv(num_agents=args['num_agents'])
            else:
                print("Can not support the " + args['env_name'] + "environment." )
                raise NotImplementedError
            env.seed(args['seed'] + rank * 1000)
            return env
        return init_env
    if args['n_rollout_threads'] == 1:
        return DummyVecEnv([get_env_fn(0)])
    else:
        return SubprocVecEnv([get_env_fn(i) for i in range(args['n_rollout_threads'])])

In [4]:
args = {'env_name':'cleanup', 'seed':2, 'n_rollout_threads':4, 'num_agents': 2}
envs = make_parallel_env(args)

In [3]:
import numpy as np 
a = np.arange(32).reshape(4,8)
a = np.expand_dims(a, 1).repeat(2, axis=1)
a

array([[[ 0,  1,  2,  3,  4,  5,  6,  7],
        [ 0,  1,  2,  3,  4,  5,  6,  7]],

       [[ 8,  9, 10, 11, 12, 13, 14, 15],
        [ 8,  9, 10, 11, 12, 13, 14, 15]],

       [[16, 17, 18, 19, 20, 21, 22, 23],
        [16, 17, 18, 19, 20, 21, 22, 23]],

       [[24, 25, 26, 27, 28, 29, 30, 31],
        [24, 25, 26, 27, 28, 29, 30, 31]]])

In [5]:
import numpy as np
a = np.random.randn(10,2,3,4,4)
a.shape
c = np.concatenate([a[:,i,:,:,:] for i in range(2)], axis=1)
d = a.reshape(10,-1,4,4)
c.shape
d.shape
for i in range(10):
    for j in range(6):
        for m in range(4):
            for n in range(4):
                if c[i][j][m][n] != d[i][j][m][n]:
                    print (i,j,m,n)
        



(10, 2, 3, 4, 4)

(10, 6, 4, 4)

(10, 6, 4, 4)

In [4]:
import numpy as np 
a = np.arange(720).reshape(6,5,4,3,2)
b = a[:,0,:]
c = a[:,0,:,:,:]
b.shape
c.shape



(6, 4, 3, 2)

(6, 4, 3, 2)